#Optuna tuning for CATBoost

In [ ]:
!pip install optuna
!pip install catboost
#!pip install optcat


     |████████████████████████████████| 256kB 4.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 4.8MB/s 
     |████████████████████████████████| 163kB 7.8MB/s 
     |████████████████████████████████| 143kB 8.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 112kB 11.5MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
  Created wheel for optuna: filename=optuna-2.2.0-cp36-none-any.whl size=340928 sha256=da383a809a4bcd218c08c22b0c39b0e907cc1a01de5fa03dbd9c30e748986b83
  Stored in directory: /root/.cache/pip/wheels/a6/27/26/3ef57a8fb2c8bceef3f4ece73483cec1895e06eae442d34a41
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.1-cp36-none-any.whl size=11119 sha256=365e5606f0007cb00406175de9106f0290989b724163c79e1d629dd5ce17d4a2
  Stored in directory: /root/.cache/pip/wheels/44

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from scipy.stats import randint as sp_randint
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
import optuna
# from optcat.core import CatBoostClassifier

In [ ]:
def average(df):
    sum = 0
    count = 0
    for x in df['f1']:
        if x!=-1:
            sum+=x
            count+=1
    avg = sum/count
    return avg

In [ ]:
def results(model, X_train, X_valid,y_train, y_valid):
    soft_probs = model.predict_proba(X_valid)
    valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
    print(valid_auc)
    train_preds = model.predict(X_train)
    acc = accuracy_score(y_train, train_preds)
    print(acc)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    print(acc)
    print(classification_report(y_valid,preds))

In [ ]:
df = pd.read_csv("train_final.csv")
# f1_avg = average(df)
# df['f1'].replace(-1,f1_avg, inplace = True)
# df.describe()

In [ ]:
y = df.loc[:,'Y']
X = df.loc[:,'f1':'f24']
X['f3']=100000*X['f3']
X['f3'] = X['f3'].astype('int')
# X['f1'] = X['f1'].astype('int')
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size = 0.75, test_size = 0.25,random_state = 42, shuffle = True)

In [ ]:
X.dtypes

f1       int64
f2       int64
f3     float64
f4       int64
f5       int64
f6       int64
f7       int64
f8       int64
f9       int64
f10      int64
f11      int64
f12      int64
f13      int64
f14    float64
f15      int64
f16      int64
f17      int64
f18      int64
f19      int64
f20      int64
f21      int64
f22      int64
f23      int64
f24      int64
dtype: object

In [ ]:
# 1st best (cat_features=cat_indices,
#                            n_estimators=950,                         
#                            learning_rate =0.01600,
#                            random_state = 42,
#                            subsample = 0.66322,
#                            early_stopping_rounds = 30,
#                            max_depth = 8,
#                            one_hot_max_size = 10,
#                            verbose=100,
#                            eval_metric='AUC:hints=skip_train~false')

In [ ]:
# # cat_indices = [0,1,3,4,5,6,7,8,9,10,11,12,14,15,16,17,19,18,20,21,22,23]
# cat_indices = [0,1,3,4,5,6,7,8,9,10,11,12,14,15,16,17,19,18,20,21,22,23]
# model_cat = CatBoostClassifier(cat_features=cat_indices,
#                            n_estimators=950,                         
#                            learning_rate =0.01600,
#                            random_state = 42,
#                            subsample = 0.66322,
#                            early_stopping_rounds = 30,
#                            max_depth = 8,
#                            bagging_temperature = 10.0,
#                            one_hot_max_size = 10,
#                            verbose=100,
#                           #  task_type='GPU',
#                           #  bootstrap_type = "Poisson",
#                            eval_metric='AUC:hints=skip_train~false')
# model_cat.fit(X_train,y_train)
# results(model_cat,X_train, X_valid, y_train, y_valid)

0:	learn: 0.5608977	total: 58.6ms	remaining: 55.6s
100:	learn: 0.8754048	total: 4.97s	remaining: 41.7s
200:	learn: 0.9073232	total: 11.3s	remaining: 42s
300:	learn: 0.9249716	total: 17.4s	remaining: 37.5s
400:	learn: 0.9338899	total: 23.6s	remaining: 32.2s
500:	learn: 0.9412157	total: 29.3s	remaining: 26.3s
600:	learn: 0.9473682	total: 35.4s	remaining: 20.5s
700:	learn: 0.9527641	total: 41.8s	remaining: 14.9s
800:	learn: 0.9587952	total: 48.5s	remaining: 9.01s
900:	learn: 0.9641528	total: 55.1s	remaining: 2.99s
949:	learn: 0.9666309	total: 58.4s	remaining: 0us
0.9128588031286895
0.9779441686335151
0.965576171875
              precision    recall  f1-score   support

           0       0.90      0.42      0.57       224
           1       0.97      1.00      0.98      3872

    accuracy                           0.97      4096
   macro avg       0.93      0.71      0.78      4096
weighted avg       0.96      0.97      0.96      4096



In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
SEED = 42
cat_indices = [0,2,1,3,4,5,6,7,8,9,10,11,12,14,15,17,19,18,20,21,22,23]

In [ ]:
def objective(trial):

    params_cat_optuna = {
        # 'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.3, 1.0),
        'max_depth': trial.suggest_int('max_depth', 7, 13),
        'n_estimators' : trial.suggest_int('n_estimators',900,1500),
         'subsample': trial.suggest_uniform('subsample', 0.5, 0.9),
        'learning_rate':trial.suggest_uniform('learning_rate', 0.01, 0.08)
    }

    #update parameters
    # if params_cat_optuna['bootstrap_type'] == 'Bayesian':
    #     params_cat_optuna['bagging_temperature'] = trial.suggest_uniform('bagging_temperature', 0, 10)
    # elif params_cat_optuna['bootstrap_type'] == 'Bernoulli':
    #     params_cat_optuna['subsample'] = trial.suggest_uniform('subsample', 0.4, 1.0)
        
    # fit the model
    model = CatBoostClassifier(cat_features=cat_indices,
                               random_state=SEED, 
                               eval_metric="AUC",
                               **params_cat_optuna,
                               one_hot_max_size=10,
                               bootstrap_type = "Bernoulli"
                               )
    model.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=0,
            early_stopping_rounds=100)
    
    ypreds = model.predict(X_valid)
    ypreds = np.rint(ypreds)
    score = roc_auc_score(y_valid.to_numpy().ravel(),
                              ypreds)
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=SEED)
N_TRIALS = 100 # make it large

study = optuna.create_study(direction='maximize',
                            sampler=sampler,
                            study_name='cat_optuna',
                            storage='sqlite:///cat_optuna_fraud_detection.db',
                            load_if_exists=True)

study.optimize(objective, n_trials=N_TRIALS,timeout=1000)

In [ ]:
print(f'Number of finished trials: {len(study.trials)}')

# best trail
best_trial = study.best_trial

# best params
params_best = study.best_trial.params
params_best

Number of finished trials: 60


{'learning_rate': 0.04312410475726457,
 'max_depth': 7,
 'n_estimators': 1221,
 'subsample': 0.8414500045940871}

In [ ]:
print(params_best)

{'colsample_bylevel': 0.3785351404284387, 'learning_rate': 0.036653805837398225, 'max_depth': 7, 'n_estimators': 1144, 'subsample': 0.7938237574450032}


In [ ]:
clf = CatBoostClassifier(cat_features=cat_indices,random_state=SEED, eval_metric="AUC", one_hot_max_size=10,verbose=False)
clf.set_params(**params_best)
clf.fit(X_train, y_train)
results(clf,X_train, X_valid, y_train, y_valid)

0.9112734743949232
0.9803043867502238
0.965087890625
              precision    recall  f1-score   support

           0       0.85      0.44      0.58       224
           1       0.97      1.00      0.98      3872

    accuracy                           0.97      4096
   macro avg       0.91      0.72      0.78      4096
weighted avg       0.96      0.97      0.96      4096



In [ ]:
clf.fit(X,y)

In [ ]:
testdf = pd.read_csv("test_final.csv")

X_test = testdf.loc[:,'f1':'f24']
X_test['f3']=100000*X_test['f3']
X_test['f3'] = X_test['f3'].astype('int')
test_preds = clf.predict_proba(X_test)
substacked = {"Id":testdf['Id'],"Y":test_preds[:,1]}
substacked= pd.DataFrame(data=substacked)
substacked.to_csv("optuna2.csv", index=False)

In [ ]:
# Resume from last time
sampler = optuna.samplers.TPESampler(seed=SEED)
N_TRIALS = 2000 # make it large

study = optuna.create_study(direction='maximize',
                            sampler=sampler,
                            study_name='cat_optuna',
                            storage='sqlite:///cat_optuna_fraud_detection.db',
                            load_if_exists=True)

study.optimize(objective, n_trials=N_TRIALS,timeout=1000)

In [ ]:
print(f'Number of finished trials: {len(study.trials)}')

# best trail
best_trial = study.best_trial

# best params
params_best = study.best_trial.params
params_best

In [ ]:
clf = CatBoostClassifier(random_state=SEED, eval_metric="AUC", one_hot_max_size=10,verbose=False)
clf.set_params(**params_best)
clf.fit(X_train, y_train)
results(clf,X_train, X_valid, y_train, y_valid)